# Tweets Cleaning lab 

## By David Bautista

### In this document we are going to test a data cleaning mechanism according to the vectorial similarity that each tweet has with the constructed corpus -taxonomy-

In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
import spacy
nlp = spacy.load("es_core_news_md")

In [5]:
import pickle

In [27]:
corpus_file = open('../taxonomia/corpus_sexperto_with_stemming_lemma.pkl', 'rb')

In [28]:
corpus = pickle.load(corpus_file)

In [29]:
corpus_list = [j for sub in corpus for j in sub]

In [30]:
corpus_set = set(corpus_list)

In [31]:
# function to clean text
def tweet_to_words(raw_review):
    # 1. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review) 

    # 2. Convert to lower case and only words
    only_words = letters_only.lower()
    only_words = " ".join(only_words.split())
    # 2.1 the nlp doc and words ❤️ 
    doc = nlp(only_words)
    lexical_tokens = [tok.lemma_ for tok in doc if tok.pos_ != 'PRON' and not (tok.is_punct | tok.is_stop)]
    words = [t.lower() for t in lexical_tokens if len(t) > 2 and t.isalpha()]
    # 3. Remove Stopwords. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("spanish"))  
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]  #returns a list 
    # 4.1 if you want remove words in anti-taxonomy
    #meaningful_words = list(filter(is_in_anti_tax, meaningful_words)) 
    # 5. Stem words. Need to define porter stemmer above
    meaningful_words = [(lambda x: stemmer.stem(x) if len(x) > 5 else x)(word) for word in meaningful_words]
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( meaningful_words ) 

In [32]:
def similarity_calculator(c, t):
    i = c.intersection(t)
    return float(len(i)) / (len(c) + len(t) - len(i))

# how it works with 1 tweet? 🤔

In [47]:
tweet_list = tweet_to_words("No trates de relacionar cosas que no tienen relación, el que haya hecho cosas tontas (porque eso fueron, ni siquiera fueron graves), no quita que no merezca que le acusen de delitos tan graves, delitos que nunca cometió.")

In [48]:
list(set([1,2,3,4,4,4,4,2]))

[1, 2, 3, 4]

In [49]:
tweet_set = set(tweet_list)

In [50]:
tweet_set

{'acus',
 'cometi',
 'delit',
 'grave',
 'merec',
 'quito',
 'relaci',
 'relacion',
 'siqu',
 'tonto',
 'trat'}

In [51]:
ss = similarity_calculator(corpus_set, tweet_set)
ss

0.001076426264800861

# and with tweet in db? 🤔

In [13]:
import pymongo

In [14]:
client = pymongo.MongoClient("mongodb://54.90.49.105:27017", username='rw_admin_smdt', password='HolocaustoDel84', authSource="smdt")
db = client["smdt"]
tweets_collection = db['tweets']

In [15]:
results_table = []

In [16]:
for tweet in tweets_collection.find({}):
    #print(tweet['tweet_id'], tweet['_id'], tweet['text'], )
    tweet_list = tweet_to_words(tweet['text'])
    tweet_set = set(tweet_list)
    results_table.append({'twitter_id': tweet['tweet_id'], 'db_id': tweet['_id'], 'text': tweet['text'], 'accuracy': similarity_calculator(corpus_set, tweet_set)})

KeyboardInterrupt: 

In [47]:
len(results_table)

41879

In [48]:
res_df = pd.DataFrame(results_table)

In [40]:
res_df = pd.read_csv('tweets_similarity_1.csv', lineterminator='\n')

In [42]:
pd.set_option('display.max_colwidth', -1)
res_df.sort_values(by='accuracy', ascending=False)

,Unnamed: 0,twitter_id,db_id,text,accuracy
0,5891,984887624618921984,5ed541dbef2f5847b9ff3fd8,"Es violento q se considere el pensamiento objetor de una persona que NO está viviendo el miedo en su cuerpo o de una entidad q NO tiene conciencia porq es una cosa, antes q el derecho de la mujer d interrumpir su embarazo, porq es su cuerpo, su decisión. http://www.adnradio.cl/noticias/nacional/todos-los-medicos-de-osorno-se-niegan-a-realizar-abortos-por-causal-de-violacion/20180413/nota/3736743.aspx …",0.005796
1,2786,1227664835149803520,5ed541d2ef2f5847b9ff30ae,"""a pesar de que hay una gama de leyes nacionales e internacionales y la #NOM046 que respaldan la interrupción legal del embarazo, las mujeres siguen encontrando barreras para acceder a los servicios de #salud en la entidad"": https://www.eloccidental.com.mx/local/jalisco-no-garantiza-el-aborto-seguro-a-mujeres-victimas-de-violacion-sistema-de-salud-ipas-4823901.html … #abortolegalmx2020 #AbortoLegal",0.005550
2,35758,973255418616086528,5ed55af39da305e9846dd6c9,Descarga este inspirador libro: \nhttp://www.lluviadelove.com/es/te-regalo-mi-libro-conexiones-de-amor-para-que-entiendas-como-conectar-con-tu-bebe-en-el-utero-el-te-dara-mas-informacion-que-una-ecografia-en-4d-2016/ …\n#Embarazada #Embarazo #fertilidad #fertility #LecturaRecomendada #LibrosRecomendados #libros #maternidad #madres #Bebés #arcoiris #corazon #ValorEs #emociON #emociones #DiaDeLaFamilia #Familia #matronapic.twitter.com/ebUh7ej9bw,0.005548
3,37720,1047972706862477312,5ed55af79da305e9846de034,"Como no los escucho hablar de prevención de embarazo no deseado y no espero que un gobierno y grupos con agenda progresista lo hagan\n“Para ejercer este derecho, tanto hombres como mujeres utilizan métodos anticonceptivos para evitar embarazos no deseados”\n http://www.aprofe.org.ec/index.php/es/servicios/14-planificacion-familiar.html …",0.005314
4,22235,1101175668673515524,5ed541efef2f5847b9ff876f,"El ejercicio durante el embarazo nos ayudará a mantenernos en forma nuestro cuerpo, regular los niveles de azúcar, promueve un aumento de peso saludable, reducir el riesgo de padecer alguna lesión y favorecerá que mantengamos una correcta postura corporal. pic.twitter.com/YQQA520Q4R",0.005074
...,...,...,...,...,...
41874,1142,797218868502687746,5ed541ceef2f5847b9ff285b,EMBARAZO CREEPY | LOWE PUEDE ESTAR EMBARAZADA - esto es peor que una inv... https://youtu.be/5ryFTjx3DSg vía @YouTube,0.000241
41875,1117,16391144785,5ed541ceef2f5847b9ff2842,"#aldia Nicolle Aldana está embarazada de Mauricio Montero, su esposo // El chunche embarazó a Nicolle Aldana? XD",0.000241
41876,29312,40182497060986880,5ed541f6ef2f5847b9ffa645,@asanca aún le queda : La duración del embarazo del elefante oscila entre los dieciocho y los veintidós meses,0.000241
41877,1067,139218544855498752,5ed541ceef2f5847b9ff2810,"O sea, 7 meses de embarazo y yo me veo más redondo U_U RT @RPPNoticias Embarazada y sexy: Beyoncé luce con orgullo su nueva figura ...",0.000241


In [67]:
res_df.sort_values(by='accuracy', ascending=False).to_csv('tweets_similarity_1.csv') 

In [64]:
[tw for tw in results_table if tw['twitter_id'] == '984887624618921984']


[{'twitter_id': '984887624618921984',
  'db_id': ObjectId('5ed541dbef2f5847b9ff3fd8'),
  'text': 'Es violento q se considere el pensamiento objetor de una persona que NO está viviendo el miedo en su cuerpo o de una entidad q NO tiene conciencia porq es una cosa, antes q el derecho de la mujer d interrumpir su embarazo, porq es su cuerpo, su decisión. http://www.adnradio.cl/noticias/nacional/todos-los-medicos-de-osorno-se-niegan-a-realizar-abortos-por-causal-de-violacion/20180413/nota/3736743.aspx\xa0…',
  'accuracy': 0.005795701521371649}]